There are two whole numbers:<br>
1 < a,b <100<br>
<br>
One scientist("Sum") get provided with sum of numbers,<br>
another  ("Prod") get provided with product of numbers.<br>
Both scientists know that numbers 1 < a,b <100.<br>
<br>
Determine the numbers being based on the following dialog: <br>
    Prod: I don't know the numbers;<br>
    Sum: I know it;<br>
    Prod: then I know the numbers; <br>
    Sum: then I know the numbers too.<br>

In [87]:
# unique_prods = both numbers prime

# Prod: I don't know the numbers; >>> at least one number is not prime

# Sum: I know it; >>> cannot be a sum of prime numbers

# Prod: then I know the numbers; >>> if divided on factors there is only one option of not being a sum of prime numbers

# Sum: then I know the numbers too. >>> if divided on summands there is only one option 
#                                       that leaves Prod with only one option on previous step


# The above looks quite complicated and layered, and it seems that there will be a lot of different lists
# to be managed in process, e.g. lists for all possible products, sums, sets of factors, prime numbers.
# But thinking of a way to make the solution more readable and efficient, using numpy matrices comes to mind.
# If we create one matrix for products and one for sums, where the indices correspond to the numbers in range(2,100),
# then we can conveniently exclude the options from both matrices by indices or values, step by step.
# I assume that when we complete all the steps, there will be only one pair of indices left in both matrices.
# And that combination of numbers will be the answer.

# !!!!!!!!!the below is to be deleted
# long story short:
# for prod, if divided on factors there is only one option of not being a sum of prime numbers

# so there should exist only one product (not only one)
# which doesn't consist of primes only,
# factors of which can only be split in one way
# in order to form a sum which can only be formed by non prime numbers

# and then there is only one sum that provides only one option for product

In [88]:
import numpy as np
import itertools

def list_primes(upto):
    odds = np.arange(3, upto + 1, 2)
    isprime = np.ones((upto - 1) // 2, dtype=bool)
    for factor in odds[:int(upto**.5)//2]:
        if isprime[(factor-2)//2]:
            isprime[(factor*3-2)//2::factor] = 0
    return np.insert(odds[isprime], 0, 2)

primes = np.array(list_primes(99))
numbers = np.arange(2, 100)

products = np.zeros((100, 100), dtype=int)
products[2:, 2:] = numbers[:, None] * numbers[None, :]

sums = np.zeros((100, 100), dtype=int)
sums[2:, 2:] = numbers[:, None] + numbers[None, :]

print(products, '\n')
print(sums)

[[   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    4 ...  194  196  198]
 ...
 [   0    0  194 ... 9409 9506 9603]
 [   0    0  196 ... 9506 9604 9702]
 [   0    0  198 ... 9603 9702 9801]] 

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   4 ...  99 100 101]
 ...
 [  0   0  99 ... 194 195 196]
 [  0   0 100 ... 195 196 197]
 [  0   0 101 ... 196 197 198]]


In [99]:
def remove_diagonal(x, y):
    '''Removes the whole opposite diagonal from products and sums matrices'''
    rng = np.arange(max(x + y - 99, 0), min(x + y + 1, 100))
    sums[rng, rng[::-1]] = 0
    products[rng, rng[::-1]] = 0

def available_values():
    '''Returns indices of available values for the next step'''
    return np.nonzero(products)

def number_available():
    '''Returns the number of available values for the next step'''
    return np.sum(products > 0)

'Total available pairs of numbers: ', number_available()

('Total available pair of numbers: ', 3856)

In [100]:
# Prod: I don't know the numbers; >>> at least one number is not prime
# Step 1. Exclude all values where both numbers are prime.

# Sum: I know it; >>> cannot be a sum of prime numbers
# Step 2. Exclude all values that can be formed by sum of prime numbers
# i.e exclude opposite diagonal for each where value from Step 1.

prime_combos = itertools.combinations_with_replacement(primes, 2)

for combo in prime_combos:
    remove_diagonal(*combo)

'Available pairs of numbers after Step 1 and Step 2: ', number_available()

('Available pairs of numbers after Step 1 and Step 2: ', 3856)

In [91]:
import numpy as np
from itertools import combinations
import primefac

def list_primes(upto):
    odds = np.arange(3, upto + 1, 2)
    isprime = np.ones((upto - 1) // 2, dtype=bool)
    for factor in odds[:int(upto**.5)//2]:
        if isprime[(factor-2)//2]:
            isprime[(factor*3-2)//2::factor] = 0
    return np.insert(odds[isprime], 0, 2)

numbers = np.arange(2, 100)
primes = np.array(list_primes(99))
print(primes)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [92]:
number_combos = list(combinations(numbers, 2)) # but actually numbers can be equal too
prime_combos = list(combinations(primes, 2))

In [93]:
number_prods = {c[0] * c[1] for c in number_combos}
prime_prods = {c[0] * c[1] for c in prime_combos}
unprime_prods = number_prods - prime_prods

print(len(number_prods), len(prime_prods))
print(len(unprime_prods))

2783 300
2483


In [94]:
number_sums = {c[0] + c[1] for c in number_combos} # just all numbers up to 99 + 99 AND same for products but without primes
prime_sums = {c[0] + c[1] for c in prime_combos}
unprime_sums = number_sums - prime_sums

print(len(number_sums), len(prime_sums))
print(len(unprime_sums))

193 109
84


In [95]:
# # all prods that can be divided in one way only to form a prime sum
# for prod in unprime_prods:
#     factors = list(primefac.primefac(prod))
#     factor_combos = [set(combinations(factors, i)) for i in range(1, len(factors))]
#     factor_pairs = {(combo, factor - combo ...) for l in factor_combos for combo in l} # need numpy too much
#     pair_sums = ...
#     if sum(pair_sums == unprime_sums) == 1:
#         ... # then this prod is good
#     print(factors)
#     print(factor_combos)
#     break